# Правильный файл

In [1]:
import pandas as pd
import numpy as np
import os

from model import get_data

from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


In [2]:
data = get_data('data/cleaned_data.csv')
predictions = pd.read_csv('data/cleaned_predict.csv', header=None).values.astype('int64').astype('str')

(456770, 3)
(456441, 3)


In [3]:
y_true = data[:, 1]
y_true = np.where(y_true == '-1', '0', y_true)

confusion_matrix(y_true, predictions)

array([[  3568,   5737,  94377],
       [  2850,   6907,  97514],
       [  4685,   7622, 233181]])

In [4]:
class_weights = compute_class_weight('balanced', np.unique(y_true), y_true)
class_weights = {0: class_weights[0],
                 1: class_weights[1],
                 2: class_weights[2]}

In [6]:
# coding=utf-8
import string

import keras
from keras.layers import Bidirectional, Embedding
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import pandas as pd
import numpy as np
import data_util
import io
from keras.utils.np_utils import to_categorical
import math
from functools import reduce
from sklearn.metrics import precision_score, recall_score

max_features = 20000
max_len = 20
batch_size = 32
epochs = 6
cur_model_name = 'multiclass_bidir_2layer_16_emb_100_drop02everywhere3_4.h5'

def train_model():
    data = get_data('data/cleaned_data.csv')
    np.random.seed(17)
    np.random.shuffle(data)

    xs, _ = get_xs(data)
    best_words_set = data_util.construct_good_set(xs, max_features, 0)
    data_util.sentences_to_scalars(xs, best_words_set)

    train_test_split = int(0.8 * len(xs))

    xs_train_scalar = keras.preprocessing.sequence.pad_sequences(xs[:train_test_split], maxlen=max_len, padding='post',
                                                                 truncating='post')
    xs_test_scalar = keras.preprocessing.sequence.pad_sequences(xs[train_test_split:], maxlen=max_len, padding='post',
                                                                truncating='post')

    ys_train, ys_test = get_ys(data, train_test_split)
    ys_train = to_categorical(ys_train)
    ys_test = to_categorical(ys_test)
    print(ys_train[:5,:])

    model = construct_model(max_features, max_len)

    model.fit(
        xs_train_scalar, ys_train,
        batch_size=batch_size,
        validation_data=(xs_test_scalar, ys_test),
        epochs=epochs,
        class_weight=class_weights
    )

    model.save(cur_model_name)


def get_xs(data):
    xs_str = data[:, 0]
    xs = np.copy(xs_str)
    xs = [str(x).encode().decode('utf-8').split(' ') for x in xs]
    return xs, xs_str

def test_model_on_unlabeled_data():
    data = get_data('data/cleaned_data_ok.csv')

    xs, xs_str = get_xs(data)
    best_words_set = data_util.construct_good_set(xs, max_features, 0)

    data_util.sentences_to_scalars_loaded_dict(xs, best_words_set)

    xs_scalar = keras.preprocessing.sequence.pad_sequences(xs, maxlen=max_len, padding='post',
                                                                 truncating='post')

    model = load_model(cur_model_name)
    result = model.predict(xs_scalar)
    max_indices = [np.argmax(r) for r in result]
    # strings_and_scores = zip(xs_str, result)
    pos = []
    neg = []
    neutral = []

    for index, i in enumerate(max_indices):
        if i == 0:
            neg.append((xs_str[index], result[index]))
            continue
        if i == 1:
            pos.append((xs_str[index], result[index]))
            continue
        if i == 2:
            neutral.append((xs_str[index], result[index]))
            continue
        print("Something's wrong")
    pos = list(sorted(pos, key=lambda x: x[1][1]))
    neg = list(sorted(neg, key=lambda x: x[1][0]))
    neutral = list(sorted(neutral, key=lambda x: x[1][2]))

    # visual_sorted = list(sorted(strings_and_scores, key=lambda x:x[1]))
    def write_to_file(strings_and_scores, path):
        with io.open(path, 'w', encoding='utf-8') as file_handler:
            for item in strings_and_scores:
                st = str(item[0]).decode('utf-8')
                file_handler.write(u"{}\t{}\n".format(st, item[1]))

    write_to_file(pos, 'data/res_pos.txt')
    write_to_file(neg, 'data/res_neg.txt')
    write_to_file(neutral, 'data/res_neutral.txt')
    print(3)


def get_ys(data, train_test_split):
    ys = data[:, 1]
    ys = np.where(ys == '-1', '0', ys)
    
    '''
    identity = str.maketrans('', '')
    allchars = ''.join(chr(i) for i in range(256))
    nondigits = allchars.translate(identity, string.digits + '-')
    # 
    #this code below is very ugly (it was build around edge cases), but it works.
    ys = [str(y).translate(identity, nondigits) for y in ys]
    ys = [y if len(y) > 0 else '2' for y in ys]
    ys = [0 if y == -1 else int(
        2 if y == '-' or (len(y) > 1 and y[1] == '-') or math.isnan(float(y))
            else (0 if y == '-1' else y)#.translate(identity, nondigits)
    ) for y in ys]
    '''
    
    ys_train = ys[:train_test_split]
    ys_test = ys[train_test_split:]
    return ys_train, ys_test


def get_data(data_path):
    df = pd.read_csv(data_path, delimiter=",")
    print(df.shape)
    #print(df['label'].unique())
    df = df[df['label'].isin(['-1', '1', '0', '2'])]
    print(df.shape)
    data = df.values[:, 1:]
    return data


def construct_model(max_features, max_len):
    model = Sequential()

    model.add(Embedding(max_features, 100, input_length=max_len))
    model.add(Bidirectional(LSTM(16, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    model.add(Bidirectional(LSTM(16, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    print(model.summary())
    return model

#train_model()
#test_model_on_unlabeled_data()



In [28]:
cur_model_name = 'multiclass_bidir_2layer_16_emb_100_drop02everywhere3_1.h5'

def test_model_on_labeled_data(data):
    data = get_data(data)

    xs, xs_str = get_xs(data)
    best_words_set = data_util.construct_good_set(xs, max_features, 0)
    data_util.sentences_to_scalars_loaded_dict(xs, best_words_set)

    xs_scalar = keras.preprocessing.sequence.pad_sequences(xs, maxlen=max_len, padding='post',
                                                                 truncating='post')

    ys = data[:, 1]
    #print(np.unique(ys.astype('str')))
    ys = [0 if y == '-1' else int(0 if math.isnan(float(y)) else y) for y in ys]
    model = load_model(cur_model_name)
    print(xs_scalar.shape)
    print(xs_scalar[:1,:], model.predict(xs_scalar[:1,:]))
    result = np.argmax(model.predict(xs_scalar), axis=1)
    np.savetxt('data/cleaned_predict_test_1.csv', result, delimiter=',')
    #print(result)
    '''
    visual = zip(xs_str, result)
    predicted_and_true = zip(result, ys)
    true_pos = reduce(lambda a, b: a + (1 if b[0] == b[1] and b[0] == 1 else 0), predicted_and_true, 0)
    false_pos = reduce(lambda a, b: a + (1 if b[0] != b[1] and b[1] == 0 else 0), predicted_and_true, 0)
    false_neg = reduce(lambda a, b: a + (1 if b[0] != b[1] and b[1] == 1 else 0), predicted_and_true, 0)
    
    precision = true_pos / np.float(true_pos + false_neg)
    recall = true_pos / np.float(true_pos + false_pos)
    #precision = precision_score(ys, result)
    #recall = recall_score(ys, result)
    print(precision)
    print(recall)
    '''
    
test_model_on_labeled_data('data/cleaned_data_test.csv')

(91289, 3)
(91289, 3)
(91289, 20)
[[ 208 5409 8757 1017   25 1358    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]] [[0.04227379 0.9492497  0.00847657]]


# Making train and test datasets

In [27]:
data = get_data('data/cleaned_data.csv')
np.random.seed(17)
np.random.shuffle(data)

#best_words_set = data_util.construct_good_set(xs, max_features, 0)
#data_util.sentences_to_scalars(xs, best_words_set)

train_test_split = int(0.8 * data.shape[0])

train_data = data[:train_test_split,:]
test_data = data[train_test_split:,:]

train_data = pd.DataFrame(train_data, columns=['text', 'label'])
test_data = pd.DataFrame(test_data, columns=['text', 'label'])

train_data.to_csv('data/cleaned_data_train.csv')
test_data.to_csv('data/cleaned_data_test.csv')

(456770, 3)
(456441, 3)


## Анализ самой первой модели

In [22]:
data = get_data('data/cleaned_data.csv')
predictions = pd.read_csv('data/cleaned_predict.csv', header=None).values.astype('int64').astype('str')

y_true = data[:, 1]
y_true = np.where(y_true == '-1', '0', y_true)

confusion_matrix(y_true, predictions)

(456770, 3)
(456441, 3)


array([[  3568,   5737,  94377],
       [  2850,   6907,  97514],
       [  4685,   7622, 233181]])

In [26]:
acc = np.sum(np.diag(confusion_matrix(y_true, predictions))) / y_true.shape[0]
acc

0.5338170760295416

## Анализ модели номер 1 - тренируем с поправкой на классовые веса в течение 10 эпох

In [28]:
data = get_data('data/cleaned_data.csv')
predictions = pd.read_csv('data/cleaned_predict_1.csv', header=None).values.astype('int64').astype('str')

y_true = data[:, 1]
y_true = np.where(y_true == '-1', '0', y_true)

confusion_matrix(y_true, predictions)

(456770, 3)
(456441, 3)


array([[ 83081,  12259,   8342],
       [ 11373,  85006,  10892],
       [ 27400,  34636, 183452]])

In [29]:
acc = np.sum(np.diag(confusion_matrix(y_true, predictions))) / y_true.shape[0]
acc

0.7701740203005427

## Анализ модели номер 2 - немного апгрейдим batch_size, lr и количество эпох=15

In [30]:
data = get_data('data/cleaned_data.csv')
predictions = pd.read_csv('data/cleaned_predict_2.csv', header=None).values.astype('int64').astype('str')

y_true = data[:, 1]
y_true = np.where(y_true == '-1', '0', y_true)

confusion_matrix(y_true, predictions)

(456770, 3)
(456441, 3)


array([[ 73940,  19573,  10169],
       [ 21329,  71334,  14608],
       [ 38983,  50061, 156444]])

In [31]:
acc = np.sum(np.diag(confusion_matrix(y_true, predictions))) / y_true.shape[0]
acc

0.6610230018775701

## Анализ модели номер 3 - как модель 3, но количество эпох=8

In [36]:
data = get_data('data/cleaned_data.csv')
predictions = pd.read_csv('data/cleaned_predict_3.csv', header=None).values.astype('int64').astype('str')

y_true = data[:, 1]
y_true = np.where(y_true == '-1', '0', y_true)

confusion_matrix(y_true, predictions)

(456770, 3)
(456441, 3)


array([[ 71056,  21488,  11138],
       [ 21003,  70140,  16128],
       [ 36869,  51033, 157586]])

In [37]:
acc = np.sum(np.diag(confusion_matrix(y_true, predictions))) / y_true.shape[0]
acc

0.6545906261707428

# Новые эмбеддинги

https://rusvectores.org/ru/about/



In [13]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('data/news_upos_cbow_300_2_2017.bin', binary=True)
model.save_word2vec_format('data/news_upos_cbow_300_2_2017.txt', binary=False)

In [14]:
model